# Инициализация

In [2]:
import numpy as np
import pandas as pd
import os
import re

In [3]:
import sys

from google.colab import drive
drive.mount('/content/drive')

ROOT = "/content/drive/MyDrive/projects-ds/birds/"
sys.path.append(os.path.join(ROOT, 'code'))

Mounted at /content/drive


In [4]:
from bird_logger import Logger, LogOperation
from bird_progress import ProgressBox
from bird_files import Bird

# Подготовка файла охоты
Мэпинг областей-районов из файла охоты на стандартные области-районы

## Мэпинг областей

In [5]:
hu_file = os.path.join(ROOT, "joins/hunting_original.csv")
hu = pd.read_csv(hu_file)
hu.oblast = hu.oblast.str.replace("ё", "е")
hu.raion = hu.raion.str.replace("ё", "е")

raion_oblast = pd.read_csv(os.path.join(ROOT, "joins/raion_oblast.csv"))
oblast = pd.Series(raion_oblast.oblast.unique())

manual = {
    'Башкирия': "Башкортостан",
    'Коми-Пермяцкий АО': "Пермский край",
    'Пермская обл.': "Пермский край",
    'Югра-Ханты-Мансийкий АО': "Ханты-Мансийский автономный округ - Югра",
    'Ненецкий А.О.': "Ненецкий автономный округ"
}
found = {}

for obl in sorted(hu.oblast.unique()):
    if obl in manual:
        continue
    name = obl.split()[0]
    true_oblast = oblast[oblast.str.contains(name)]
    if true_oblast.shape[0]:
        found[obl] = true_oblast.values[0]

found.update(manual)

true_oblast = pd.Series(found).to_frame().rename(columns={0:"true_oblast"})

print("До пришивки областей   :", hu.shape)
hu_obl = hu.join(true_oblast, on="oblast")
print("После пришивки областей:", hu_obl.shape)
assert not any(hu_obl.true_oblast.isnull()), "ВНИМАНИЕ! Есть непришитые области!"
print("Всё ок, все области замапились")

До пришивки областей   : (9712, 7)
После пришивки областей: (9712, 8)
Всё ок, все области замапились


## Мэпинг районов

In [6]:
raion_map = {}
not_found = []

manual = {
    ("Липецкая область", "Л - Толстовский (Липецкая обл.)"): "Лев-Толстовский район",
    ('Марий Эл', 'Мари - Турекский (Марий Эл)'): "Мари-Турекский район",
    ('Марий Эл', 'Келимарский (Марий Эл)'): "Килемарский район",
    ('Татарстан', 'Р.Слободский (Татарстан)'): "Рыбно-Слободский район",
    ('Самарская область', 'Ч-Вершинский (Самарская обл.)'): "Челно-Вершинский район",
    ('Самарская область', 'Б-Черниговский (Самарская обл.)'): "Большечерниговский район",
    ('Самарская область', 'Б-Глушицкий (Самарская обл.)'): "Большеглушицкий район",
    ('Самарская область', 'К-Черкасский (Самарская обл.)'): "Кинель-Черкасский район",
    ('Пензенская область', 'Н-Ломовский (Пензенская обл.)'): "Нижнеломовский район",
    ('Пензенская область', 'М-Сердобинский (Пензенская обл.)'): "Малосердобинский район",
    ('Пензенская область', 'Кондольский (Пензенская обл.)'): "Пензенский район",
    ('Пензенская область', 'Б-Демьяновский (Пензенская обл.)'): "Спасский район",
    ('Пензенская область', 'Наровчатовский (Пензенская обл.)'): "Наровчатский район",
    ('Вологодская область', 'Кичмен.-Городецкий (Вологодская обл.)'): "Кичменгско-Городецкий район",
    ('Архангельская область', 'В-Тоемский (Архангельская обл.)'): "Верхнетоемский район",
    ('Чувашия', 'Марпосадский (Чувашия)'): "Мариинско-Посадский район",
    ('Курская область', 'Б.Солдатский (Курская обл.)'): "Большесолдатский район",
    ('Ивановская область', 'Гаврилово - Посадский (Ивановская обл.)'): "Гаврилово-Посадский район",
    ('Калмыкия', 'Ики - Бурульский (Калмыкия)'): "Ики-Бурульский район",
    ('Ульяновская область', 'Старо-Кулаткинский (Ульяновская обл.)'): "Старокулаткинский район",
    ('Удмуртия', 'Як-Бодьинский (Удмуртия)'): "Якшур-Бодьинский район",
    ('Рязанская область', 'Новодеревенский (Рязанская обл.)'): "Александро-Невский район",
    ('Свердловская область', 'мМО "Поселок Уральский"'): "городской округ Уральский",
    ('Свердловская область', 'Кувшинский ГО'): "Кушвинский городской округ",
    ('Свердловская область', 'ГО ЗАТО Свободный'): "ЗАТО Свободный",
    ('Омская область', 'Азовский'): "Азовский немецкий национальный район",
    ('Московская область', 'Богородский'): "Ногинский район",
    ('Свердловская область', 'горд Ирбит'): "городской округ Ирбит",
    ('Свердловская область', 'Ирбитский МО'): "городской округ Ирбитское",
    ('Тульская область', 'Ленинский (Тульская обл.)'): "городской округ Тула",
    ('Челябинская область', 'г.Пласт (Челябинская обл.)'): "Пластовский район",
    ('Свердловская область', 'Алапаевский (Свердловская обл.)'): "городской округ Алапаевск",
    ('Свердловская область', 'В.Пышменский (Свердловская обл.)'): "городской округ Верхняя Пышма",
    ('Свердловская область', 'Пригородный (Свердловская обл.)'): "Горноуральский городской округ",
    ('Свердловская область', 'Н.Тура (Свердловская обл.)'): "Нижнетуринский городской округ",
    ('Орловская область', 'Новодеревенский (Орловская обл.)'): "Новодеревеньковский район",
    ('Волгоградская область', 'Михайловский (Волгоградская обл.)'): "городской округ Михайловка",
    ('Псковская область', 'Стругокрасненский (Псковская обл.)'): "Струго-Красненский район",
    ('Мурманская область', 'Апатито-Кировский (Мурманская обл.)'): "городской округ Апатиты",
    ('Тульская область', 'Новомосковский. (Тульская обл.)'): "городской округ Новомосковск",
    ('Калужская область', 'Жиздринскийй (Калужская обл.)'): "Жиздринский район",
    ('Нижегородская область', 'Шахунский (Нижегородская обл.)'): "городской округ Шахунья",
    ('Нижегородская область', 'Кулебакский (Нижегородская обл.)'): "городской округ Кулебаки",
    ('Нижегородская область', 'Выксунский (Нижегородская обл.)'): "городской округ Выкса",
    ('Нижегородская область', 'Княгинский (Нижегородская обл.)'): "Княгининский район",
    ('Московская область', 'Серебряно-Прудский (Московская обл.)'): "городской округ Серебряные Пруды",
    ('Московская область', 'Шаховской (Московская обл.)'): "городской округ Шаховская",
    ('Московская область', 'Домодедовский (Московская обл.)'): "городской округ Домодедово",
    ('Пермский край', 'Гремячинск (Пермская обл.)'): "Гремячинский район",
    ('Тверская область', 'Кесовогородский (Тверская обл.)'): "Кесовогорский район",
    ('Республика Карелия', 'г.Костомукша (Карелия)'): "Костомукшский городской округ",
    ('Республика Коми', 'Воркутинский (Коми)'): "городской округ Воркута",
    ('Республика Коми', 'Вуктыльский (Коми)'): "район Вуктыл",
    ('Республика Коми', 'Интинский (Коми)'): "городской округ Инта",
    ('Республика Коми', 'Печорский (Коми)'): "район Печора",
    ('Республика Коми', 'Сосногорский (Коми)'): "район Сосногорск",
    ('Республика Коми', 'Ухтинский (Коми)'): "городской округ Ухта",
    ('Пермский край', 'Чусовой (Пермская обл.)'): "Чусовской район",
    ('Пермский край', 'Кизел (Пермская обл.)'): "Кизеловский район",
    ('Пермский край', 'Краснокамск (Пермская обл.)'): "Краснокамский район",
    ('Пермский край', 'Лысьва (Пермская обл.)'): "Лысьвенский городской округ",
    ('Московская область', 'Каширский (Московская обл.)'): "городской округ Кашира",
    ('Владимирская область', 'Вязниковскийй (Владимирская обл.)'): "Вязниковский район",
    ('Белгородская область', 'Алексеевский (Белгородская обл. )'): "Алексеевский район и город Алексеевка",
    ('Белгородская область', 'Шебекинский (Белгородская обл. )'): "Шебекинский район и город Шебекино",
    ('Ивановская область', 'Верхне - Ландеховский (Ивановская обл.)'): "Верхнеландеховский район",
    ('Вологодская область', 'Усть-Кубенский (Вологодская обл.)'): "Усть-Кубинский район",
    ('Ивановская область', 'Юрьевский (Ивановская обл.)'): "Юрьевецкий район",
    ('Владимирская область', 'Судогский (Владимирская обл.)'): "Судогодский район",
    ('Кировская область', 'Даровский (Кировская обл.)'): "Даровской район",
    ('Кировская область', 'Вятско-Полянский (Кировская обл.)'): "Вятскополянский район",
    ('Архангельская область', 'Новоземельский (Архангельская обл.)'): "городской округ Новая Земля",
    ('Марий Эл', 'Новоторьяльский (Марий Эл)'): "Новоторъяльский район",
    ('Московская область', 'Озерский (Московская обл.)'): "городской округ Озеры",
    ('Московская область', 'Люберцы'): "Люберецкий район",
    ('Московская область', 'Звездный'): "городской округ Звездный городок",
    ('Московская область', 'Фрязевский'): "городской округ Фрязино",
    ('Ставропольский край', 'Буденовский (Ставропольский край)'): "Буденновский район",
    ('Ульяновская область', 'Базарно-Сызганский (Ульяновская обл.)'): "Базарносызганский район",
    ('Ульяновская область', 'Теренгульский (Ульяновская обл.)'): "Тереньгульский район",
    ('Самарская область', 'Похвистненский (Самарская обл.)'): "Похвистневский район",
    ('Оренбургская область', 'Кувондыкский (Оренбургская обл.)'): "Кувандыкский район",
    ('Оренбургская область', 'Тюльчанский (Оренбургская обл.)'): "Тюльганский район",
    ('Алтайский край', 'Тюменьцевский'): "Тюменцевский район",
    ('Алтайский край', 'МО Славгород'): "городской округ Славгород",
    ('Алтайский край', 'Немецкий национальный'): "Немецкий национальный район",
    ('Курганская область', 'Альменьевский'): "Альменевский район",
    ('Красноярский край', 'Таймырский Долгано-Ненецкий'): "Таймырский Долгано-Ненецкий район",
    ('Свердловская область', 'ГО верхняя Тура'): "городской округ Верхняя Тура",
    ('Свердловская область', 'Режевский ГО'): "Режевской городской округ",
    ('Свердловская область', 'Полевский ГО'): "Полевской городской округ",
    ('Свердловская область', 'Махневский МО'): "городской округ Махневское",
    ('Свердловская область', 'МО Красноуфимский округ'): "Красноуфимский городской округ",
    ('Свердловская область', 'МО Алапаевское'): "городской округ Алапаевское",
    ('Свердловская область', 'МО "Город Каменск-Уральский"'): "городской округ Каменск-Уральский",
    ('Свердловская область', 'ГО "Город Лесной"'): "городской округ Лесной",
    ('Свердловская область', 'мМО "Город Екатеринбург"'): "городской округ Екатеринбург",
    ('Омская область', 'Нововаршарвский'): "Нововаршавский район",
    ('Челябинская область', 'Верхнейуфалейский ГО'): "Верхнеуфалейский городской округ",
    ('Челябинская область', 'Верхнейувалейский ГО'): "Верхнеуфалейский городской округ"
}

def filter_same(obl, rai):
    true_raion = raion_oblast[(raion_oblast.oblast==obl) & (raion_oblast.raion==rai)]
    return true_raion

def filter_gorod(obl, rai):
    rai = re.sub(r'\(.+\)|"', '', rai).strip()
    if rai[:2] == "г.":
        rai = rai[2:].strip()
    elif rai[:5] == "город":
        rai = rai[5:].strip()
    elif "ГО" in rai.split():
        rai = rai.replace("ГО", "").strip()
    else:
        return None

    true_raion = raion_oblast[(raion_oblast.oblast==obl) & ((raion_oblast.raion==f'{rai} городской округ') | (raion_oblast.raion==f'городской округ {rai}'))]
    if true_raion.shape[0] == 1:
        return true_raion

def filter_raion_gorokrug(obl, rai):
    rai = re.sub(r'\(.+\)|"', '', rai).strip()
    rai = rai.split()[0]

    true_raion = raion_oblast[(raion_oblast.oblast==obl) & (raion_oblast.raion==f'{rai} район')]
    if true_raion.shape[0] == 0:
        true_raion = raion_oblast[(raion_oblast.oblast==obl) & ((raion_oblast.raion==f'{rai} городской округ') | (raion_oblast.raion==f'городской округ {rai}'))]
    if true_raion.shape[0] == 0 and rai[-2:] in ["ий", "ой"]:
        true_raion = raion_oblast[(raion_oblast.oblast==obl) & (raion_oblast.raion==f'городской округ {rai[:-2]}')]
    if true_raion.shape[0] == 0 and rai[-4:] == "ский":
        true_raion = raion_oblast[(raion_oblast.oblast==obl) & (raion_oblast.raion==f'городской округ {rai[:-4]}')]
    return true_raion

filters = [filter_same, filter_gorod, filter_raion_gorokrug]
filter_stat = {"manual": len(manual)}

for (obl, rai), _ in hu_obl[["true_oblast", 'raion']].value_counts().iteritems():
    if (obl, rai) in manual:
        raion_map[(obl, rai)] = manual[(obl, rai)]
        continue

    found = False

    for flt in filters:
        true_raion = flt(obl, rai)
        if true_raion is not None and true_raion.shape[0] == 1:
            raion_map[(obl, rai)] = true_raion.iloc[0].raion
            found = True
            filter_stat[flt.__name__] = filter_stat.setdefault(flt.__name__, 0) + 1
            break
    if not found:
        not_found.append((obl, rai))

print(filter_stat)
assert not not_found, "ВНИМАНИЕ! Есть незамапленные районы!"

true_raion = pd.DataFrame([pd.Series([obl, rai, true_rai]) for (obl, rai), true_rai in raion_map.items()])
true_raion.columns = ["true_oblast", "raion", "true_raion"]

print("До пришивки районов   :", hu_obl.shape)
hu_rai = hu_obl.merge(true_raion, left_on=["true_oblast", "raion"], right_on=["true_oblast", "raion"], how="left")
print("После пришивки районов:", hu_rai.shape)
assert not any(~hu_rai.raion.isnull() & hu_rai.true_raion.isnull()), "ВНИМАНИЕ! Есть непришитые районы!"
print("Всё ок, все районы замапились")

{'manual': 100, 'filter_raion_gorokrug': 1310, 'filter_gorod': 63, 'filter_same': 3}
До пришивки районов   : (9712, 8)
После пришивки районов: (9712, 9)
Всё ок, все районы замапились


## Сохранение

In [7]:
hu_rai = hu_rai.drop(columns=["oblast", "raion"]).rename(columns={"true_oblast":"oblast", "true_raion":"raion"})
hu_rai.to_csv(os.path.join(ROOT, "joins/hunting.csv") ,index=False)

# Пришивка охоты к датасету птиц с уже пришитыми областями-районами

## Код пришивки флага охоты к файлу птицы

In [8]:
class Hunting:
    def __init__(self):
        self._load_from_file()

    def _load_from_file(self):
        self.df = pd.read_csv(os.path.join(ROOT, "joins/hunting.csv"), parse_dates=["open", "close"], low_memory=False)
        # делаем диапазоны широт там, где указана только min или max широта
        self.df.loc[~self.df.min_lat.isnull() & self.df.max_lat.isnull(), "max_lat"] = 90
        self.df.loc[self.df.min_lat.isnull() & ~self.df.max_lat.isnull(), "min_lat"] = 0

    def join_to_bird(self, bird:Bird):
        # 1:когда есть область-район, без широт
        bird_hunt1 = bird.df.merge(self.df.query("raion==raion and min_lat!=min_lat"),
                                left_on=["oblast", "raion"], right_on=["oblast", "raion"])
        bird_hunt1 = bird_hunt1.query("open<=timestamp and close>=timestamp")
        assert not any(bird_hunt1.event_id.duplicated()), "ВНИМАНИЕ! Есть дублирование мёржа охоты #1!"

        # 2:когда есть область-район и диапазон широт
        bird_hunt2 = bird.df.merge(self.df.query("raion==raion and min_lat==min_lat"),
                                left_on=["oblast", "raion"], right_on=["oblast", "raion"])
        bird_hunt2 = bird_hunt2.query("open<=timestamp and close>=timestamp and location_lat>=min_lat and location_lat<=max_lat")
        assert not any(bird_hunt2.event_id.duplicated()), "ВНИМАНИЕ! Есть дублирование мёржа охоты #2!"

        # 3:когда есть есть только область и диапазон широт
        bird_hunt3 = bird.df.merge(self.df.query("raion!=raion and min_lat==min_lat"),
                                on=["oblast"])
        bird_hunt3 = bird_hunt3.query("open<=timestamp and close>=timestamp and location_lat>=min_lat and location_lat<=max_lat")
        assert not any(bird_hunt3.event_id.duplicated()), "ВНИМАНИЕ! Есть дублирование мёржа охоты #3!"

        bird_hunt = bird_hunt1.append([bird_hunt2, bird_hunt3])[["event_id"]]
        assert not any(bird_hunt.event_id.duplicated()), "ВНИМАНИЕ! Есть дублирование мёржа охоты!"
        bird_hunt["hunting"] = True

        with LogOperation(LOG, bird, "join_hunting"):
            bird.df = bird.df.merge(bird_hunt, on="event_id", how="left")

        bird.df.loc[bird.df.hunting.isnull(), "hunting"] = False 
        LOG.counter_info(file, "hunting", bird.df.query("hunting==True")[["oblast", "raion"]].value_counts(dropna=False).to_dict())

## Цикл пришивки охоты к птицам **03_region -> 04_hunting**

In [11]:
%%time

BIRD_DIR = os.path.join(ROOT, "results/03_region")
LOG_DIR = os.path.join(ROOT, "logs")
SAVE_DIR = os.path.join(ROOT, "results/04_hunting")

hunting = Hunting()

LOG = Logger(LOG_DIR, prefix="hunting_")

!rm -rf $SAVE_DIR/*.*

bird_files = sorted(os.listdir(BIRD_DIR))

disp = ProgressBox(stages={'hunting':'пришиваем охоту'}, total_files=len(bird_files))

for i, file in enumerate(bird_files):
    disp.new_file(file, i+1)
    bird = Bird(os.path.join(BIRD_DIR, file))
    LOG.operation_info(bird.name, 'load', (0,0), bird.df.shape)

    disp.new_stage('hunting')
    hunting.join_to_bird(bird)

    disp.new_stage('save')
    bird.save(SAVE_DIR, prefix='h')

    disp.new_stage('ok')

LOG.flush()

CPU times: user 1min 20s, sys: 15.5 s, total: 1min 36s
Wall time: 1min 37s


## Обработка логов пришивки охоты

In [12]:
LOG_DIR = os.path.join(ROOT, "logs")

oper = pd.read_csv(os.path.join(LOG_DIR, "hunting_operations.csv"))
pi = oper.pivot(index="bird_file", columns="operation", values="shape_after").applymap(lambda sh:eval(sh)[0]).reset_index()
pi = pi[[pi.columns[2], pi.columns[1]]]
assert not any(pi.load!=pi.join_hunting), "ВНИМАНИЕ! Есть дублирование при мёрже"
print("Всё ок, дублирования при мёрже не было")

Всё ок, дублирования при мёрже не было


In [14]:
ra_log = pd.read_csv(os.path.join(ROOT, "logs/region_counters.csv"))
rus = sum(eval(cnt.replace("nan", "\"nan\""))["Россия"] for cnt in ra_log.query("counter=='country_russia'").data.values)

hu_log = pd.read_csv(os.path.join(ROOT, "logs/hunting_counters.csv"))
hunt = sum(sum(eval(cnt).values()) for cnt in hu_log.data)

print(f"Охота занимает {hunt/rus*100:.2f}% точек для всей России")

Охота занимает 8.11% точек для всей России
